# load libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import argparse
import re
import glob
from bs4 import BeautifulSoup
import requests

# crawl the website to get the data

In [ ]:
import requests
from bs4 import BeautifulSoup

# 크롤링할 URL
url = 'https://example.com'

# 요청 보내기
response = requests.get(url)

# 응답 상태 확인
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 예시: 모든 제목(h1) 태그 추출
    titles = soup.find_all('h1')
    for title in titles:
        print(title.text.strip())
else:
    print("웹 페이지를 불러오지 못했습니다.")
